In [ ]:
import sys
sys.path.append("..")
from model_analysis.jnk3_no_ask1 import model
import numpy as np
from pysb.simulator import ScipyOdeSimulator
import matplotlib.pyplot as plt
from model_analysis.equilibration_function import pre_equilibration
import sympy
from collections import OrderedDict
import pandas as pd

In [ ]:
idx_pars_calibrate = [1, 5, 9, 11, 15, 17, 19, 23, 25, 27, 31, 35, 36, 37, 38, 39, 41, 43] #pydream3
rates_of_interest_mask = [i in idx_pars_calibrate for i, par in enumerate(model.parameters)]

# calibrated_pars = np.load('jnk3_noASK1_calibrated_pars_pso_1h.npy')
calibrated_pars = np.load('../model_analysis/most_likely_par_100000.npy') # most likely parameter from pydream calibration
param_values = np.array([p.value for p in model.parameters])

jnk3_initial_idxs = [47, 48, 49]
arrestin_idx = 44
kcat_idx = [36, 37]

par_set_calibrated = np.copy(param_values)
par_set_calibrated[rates_of_interest_mask] = 10 ** calibrated_pars

In [ ]:
def plot_arrestin_noarrestin_ppjnk3():
    # Pre-equilibration
    exp_data = pd.read_csv('../data/exp_data_3min.csv')
    tspan = np.linspace(0, exp_data['Time (secs)'].values[-1], 121)
    solver = ScipyOdeSimulator(model, tspan=tspan)

    pars1 = np.copy(par_set_calibrated)
    pars2 = np.copy(par_set_calibrated)

    # Pre-equilibration
    time_eq = np.linspace(0, 100, 100)
    pars_eq1 = np.copy(par_set_calibrated)
    pars_eq2 = np.copy(par_set_calibrated)

    pars_eq2[arrestin_idx] = 0
    pars_eq2[jnk3_initial_idxs] = [0.5958, 0, 0.0042]

    all_pars = np.stack((pars_eq1, pars_eq2))
    all_pars[:, kcat_idx] = 0  # Setting catalytic reactions to zero for pre-equilibration
    eq_conc = pre_equilibration(model, time_eq, all_pars)[1]

    # Simulating models with initials from pre-equilibration and parameters for condition with/without arrestin
    pars2[arrestin_idx] = 0
    pars2[jnk3_initial_idxs] = [0.5958, 0, 0.0042]
    sim = solver.run(param_values=[pars1, pars2], initials=eq_conc).all


    print ('Fold change:',(sim[0]['all_jnk3'][-1] ) / (sim[1]['all_jnk3'][-1] ))

    plt.plot(tspan, sim[0]['all_jnk3'], color='r', label='ppJNK3 with Arrestin-3')
    plt.plot(tspan, sim[1]['all_jnk3'], color='k', label='ppJNK3 no Arrestin-3')
    plt.xlabel('Time (s)')
    plt.ylabel(r' Normalized Concentration')
    plt.legend()

In [ ]:
plot_arrestin_noarrestin_ppjnk3()